In [1]:
import pandas as pd

distribution_centers = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/distribution_centers_optimized.parquet')
inventory_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/inventory_items_optimized.parquet')
orders = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/orders_optimized.parquet')
order_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/order_items_optimized.parquet')
products = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/products_optimized.parquet')
users = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/users_optimized.parquet')
# events1 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet")
# events2 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet")
# events = pd.concat([events1, events2]).sort_index()
def events_read():    
    events1 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided1.parquet")
    events2 = pd.read_parquet("../dataset/Looker Ecommerce BigQuery Dataset/Optimization/events_optimized_divided2.parquet")
    events = pd.concat([events1, events2]).sort_index()
    return events
events = events_read()

# 1

In [19]:
events[events['event_created_at'] < "2023-12-01"]['event_created_at'].max()

Timestamp('2023-11-30 23:59:32+0000', tz='UTC')

In [18]:
order_items[(order_items['status']!="Returned") & (order_items['status'] != "Cancelled") & (order_items['created_at'] < "2023-12-01")]['created_at'].max()

Timestamp('2023-11-30 23:58:54+0000', tz='UTC')

In [5]:
len(events)

2431963

In [14]:
events['event_type'].unique()

['cancel', 'cart', 'department', 'home', 'product', 'purchase']
Categories (6, object): ['cancel', 'cart', 'department', 'home', 'product', 'purchase']

In [15]:
(events[(events['event_created_at'] > (events['event_created_at'].max() - pd.Timedelta(days=270)))]
 .dropna(subset='user_id')['user_id'].unique())

<IntegerArray>
[13199, 46234, 65826,   676, 93722, 98402,  9787, 11679,  7681, 79605,
 ...
 81639, 47458,  8514, 43595, 26739, 10798, 49358,  1303, 76896, 82709]
Length: 40977, dtype: Int32

In [9]:
users['user_id'].unique()

<IntegerArray>
[  457,  6578, 36280, 60193, 64231, 72187, 22528, 34417, 50098, 63352,
 ...
 82576, 97115, 17254, 92296, 96127, 93247, 59110, 57045, 73312, 77764]
Length: 100000, dtype: Int32

In [13]:
order_items[~order_items['status'].isin(["Returned", "Cancelled"])]['user_id'].unique()

<IntegerArray>
[22551, 28215, 20165, 71954, 84693, 37023, 63821, 15553, 38312, 30232,
 ...
 35802, 44829, 47334, 47747, 47864, 48532, 59868, 60511, 62277, 20434]
Length: 66254, dtype: Int32

In [12]:
order_items[(order_items['created_at'] < '2023-12-01')&(~order_items['status'].isin(["Returned", "Cancelled"]))]['user_id'].unique()

<IntegerArray>
[22551, 71954, 84693, 37023, 63821, 15553, 30232, 53552, 64053, 41873,
 ...
 19187, 26113, 35802, 44829, 47334, 47747, 47864, 60511, 62277, 20434]
Length: 58464, dtype: Int32

In [16]:
(events[(events['event_created_at'] > (events['event_created_at'].max() - pd.Timedelta(days=270)))].dropna(subset='user_id')
 .merge(order_items[(order_items['created_at'] < '2023-12-01')&(~order_items['status'].isin(["Returned", "Cancelled"]))], 
        on='user_id', how='inner'))['user_id'].unique()

<IntegerArray>
[46234, 65826,   676, 98402,  9787, 11679,  7681, 79605, 15771, 92473,
 ...
 61691, 67996, 28195, 47458, 43595, 26739, 10798,  1303, 76896, 82709]
Length: 26798, dtype: Int32

In [ ]:
#1~270: recent
#271~ : e^recent

In [23]:
import numpy as np
print(271 < np.exp(271))
print(np.exp(271))

True
4.940883294133372e+117


In [28]:
print(10*271)

2710


In [30]:
print(50*np.log(271))
print(50*np.log(1300))

280.10594104398507
358.5059771724814


In [47]:
print(270 + np.exp(1+1))
print(270 + (1.007)**(1+(1300-270)))

277.38905609893067
1598.5690798827952


frequency 1~4 + (1~7) = 2~11  X/(Max-min)  2/9~11/9     
recentry 1~1300         1~10        
momentary 1_000~500_000 2500000 /100000

In [51]:
order_items.groupby('user_id')['product_id'].count().max()

14

In [ ]:
import numpy as np

# 이부분 상의 앞에 RFM한 부분만 따로 보관해둘지?

# GKSMS QJQ 방법론을 이용하여 각 고객의 CLV 계산
rfm = rfm.assign(G=rfm['monetary'])  # 과거 고객 가치 (Monetary를 사용)
# rfm_data = rfm[['user_id', 'monetary']].copy().rename(columns={'monetary': 'G'})  # 과거 고객 가치 (Monetary를 사용)
rfm = rfm.assign(K=rfm['monetary'] * rfm['frequency'])  # 현재 고객 가치
rfm = rfm.assign(S=rfm['monetary'] * rfm['frequency'] / rfm['recency'])  # 잔존 고객 가치
rfm = rfm.assign(M=np.log(rfm['frequency']) * rfm['monetary'])  # 모멘텀 고객 가치
# rfm = rfm.assign(S=rfm['monetary'] * rfm['frequency'] / rfm['recency'])  # 소셜 고객 가치
rfm = rfm.assign(Q=rfm['monetary'] / rfm['frequency'])  # 품질 고객 가치
rfm = rfm.assign(J=rfm['monetary'] * np.log(rfm['recency']))  # 행동 고객 가치
# rfm = rfm.assign(J=rfm['monetary'] / np.log(rfm['recency']))  # 행동 고객 가치

# 2

In [4]:
events[(events['user_id']==2)&(events['event_type']=='purchase')]

,event_id,user_id,sequence_number,session_id,event_created_at,ip_address,city,state,postal_code,browser,event_traffic_source,uri,event_type
2231787,35,2,5,0f79ab42-3110-471e-a6ff-268a08205ddd,2022-02-20 10:27:48+00:00,55.99.143.141,Franca,São Paulo,14404,Firefox,Email,/purchase,purchase
